In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import vaex

Reading in businesses file

In [75]:
df = pd.read_json('./yelp_academic_dataset_business.json/yelp_academic_dataset_business.json', lines=True)

In [76]:
df.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None


Dropping columns irrelevant to my project.  

In [77]:
df.drop(columns=['address', 'postal_code', 'latitude', 'longitude', 'attributes', 'hours'], inplace=True)

In [79]:
df.shape

(209393, 8)

In [83]:
df.dropna(inplace=True)

Seeing how many categories contain "food"

In [84]:
df[df['categories'].str.contains('Food')]

,business_id,name,city,state,stars,review_count,is_open,categories
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,Champaign,IL,4.5,5,1,"Ethnic Food, Food Trucks, Specialty Food, Impo..."
12,M_guz7Dj7hX0evS672wIwA,Chocolate Shoppe Ice Cream,Middleton,WI,3.5,6,1,"Desserts, Food, Ice Cream & Frozen Yogurt"
20,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,Charlotte,NC,3.0,5,0,"Food, Restaurants, Grocery, Middle Eastern"
23,mKTq1T_IAplDpHUcMzOXkw,Tipsy Ryde,Gastonia,NC,3.5,3,1,"Hotels & Travel, Transportation, Taxis, Beer, ..."
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,Thornhill,ON,4.5,7,1,"Japanese, Fast Food, Food Court, Restaurants"
...,...,...,...,...,...,...,...,...
209374,BtTCTHwoIfr3fBQjaOkpIw,Starbucks,Toronto,ON,4.0,22,1,"Coffee & Tea, Food"
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,North Olmsted,OH,3.0,13,0,"Restaurants, Sandwiches, Food, Breakfast & Bru..."
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,Charlotte,NC,2.0,16,1,"Restaurants, Pizza, Sandwiches, Fast Food, Chi..."
209389,PotiAhtDMYMNsJCzV0oLAw,Walmart,Charlotte,NC,2.0,13,0,"Department Stores, Food, Mobile Phones, Fashio..."


Only keeping categories that contain "restaurant"

In [88]:
df = df[df['categories'].str.contains('Restaurant')]

In [104]:
df.reset_index(inplace=True)

In [110]:
df.drop(columns=['index'], inplace=True)

C:\Users\Luke\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [111]:
df

,business_id,name,city,state,stars,review_count,is_open,categories
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,Champaign,IL,4.5,5,1,"Ethnic Food, Food Trucks, Specialty Food, Impo..."
1,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,Charlotte,NC,3.0,5,0,"Food, Restaurants, Grocery, Middle Eastern"
2,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,Aurora,ON,4.5,4,1,"Restaurants, Cheesesteaks, Poutineries"
3,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,Thornhill,ON,4.5,7,1,"Japanese, Fast Food, Food Court, Restaurants"
4,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,Richmond Hill,ON,3.0,3,1,"Persian/Iranian, Turkish, Middle Eastern, Rest..."
...,...,...,...,...,...,...,...,...
63956,xVpE01l6ZXdEtVf5PkRpDg,Julep,Madison,WI,4.0,95,0,"Nightlife, Diners, Bars, Southern, Restaurants"
63957,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,North Olmsted,OH,3.0,13,0,"Restaurants, Sandwiches, Food, Breakfast & Bru..."
63958,hskVqZCPqy-omm9CHi44xQ,Domino's,Charlotte,NC,2.0,16,1,"Restaurants, Pizza, Sandwiches, Fast Food, Chi..."
63959,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,Vaughan,ON,4.0,5,0,"Japanese, Sushi Bars, Restaurants"


Saving as CSV rather than JSON

In [112]:
df.to_csv('./yelp_restaurants.csv')

Reading in reviews dataset

In [2]:
reviews = pd.read_json('./yelp_academic_dataset_review.json', lines=True)

In [3]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


Dropping irrelevant collumns

In [16]:
reviews.drop(columns=['review_id', 'user_id', 'date'], inplace=True)

Saving reviews as CSV rather than JSON

In [17]:
reviews.to_csv('./yelp_reviews.csv')

Reading dataset back in as CSV

In [2]:
reviews = pd.read_csv('./yelp_reviews.csv')

In [3]:
restaurants = pd.read_csv('./yelp_restaurants.csv')

In [10]:
restaurants.head()

,business_id,name,city,state,stars,review_count,is_open,categories
0,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,Champaign,IL,4.5,5,1,"Ethnic Food, Food Trucks, Specialty Food, Impo..."
1,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,Charlotte,NC,3.0,5,0,"Food, Restaurants, Grocery, Middle Eastern"
2,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,Aurora,ON,4.5,4,1,"Restaurants, Cheesesteaks, Poutineries"
3,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,Thornhill,ON,4.5,7,1,"Japanese, Fast Food, Food Court, Restaurants"
4,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,Richmond Hill,ON,3.0,3,1,"Persian/Iranian, Turkish, Middle Eastern, Rest..."


In [8]:
reviews.head()

,business_id,stars,useful,funny,cool,text
0,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I..."
1,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...
2,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...
3,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g..."
4,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas..."


In [11]:
reviews.shape

(8021122, 6)

In [12]:
restaurants['business_id'].nunique()


63961

In [13]:
reviews['business_id'].nunique()

209393

In [14]:
restaurants_to_keep = set(restaurants['business_id'])

In [22]:
reviews['keep'] = reviews['business_id'].apply(lambda x: 1 if x in restaurants_to_keep else 0)

In [23]:
reviews.head()

,business_id,stars,useful,funny,cool,text,keep
0,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",0
1,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,0
2,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,1
3,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",1
4,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",1


Double checking my work

In [29]:
restaurants.loc[restaurants['business_id'] == "IS4cv902ykd8wj1TR0N3-A"]

,business_id,name,city,state,stars,review_count,is_open,categories
2311,IS4cv902ykd8wj1TR0N3-A,Raising Cane's Chicken Fingers,Henderson,NV,3.5,159,1,"Fast Food, Restaurants, Sandwiches, Chicken Wings"


In [55]:
reviews

,business_id,stars,useful,funny,cool,text,keep
0,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",0
1,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,0
2,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,1
3,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",1
4,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",1
...,...,...,...,...,...,...,...
8021117,XW2kaXdahICaJ27A0dhGHg,1,1,0,1,"Fricken unbelievable, I ordered 2 space heater...",0
8021118,IsoLzudHC50oJLiEWpwV-w,3,1,3,1,Solid American food with a southern comfort fl...,1
8021119,kDCyqlYcstqnoqnfBRS5Og,5,15,6,13,I'm honestly not sure how I have never been to...,1
8021120,VKVDDHKtsdrnigeIf9S8RA,3,2,0,0,Food was decent but I will say the service too...,1


In [56]:
reviews = reviews[reviews['keep'] == 1]

In [59]:
reviews.reset_index(inplace=True)

In [60]:
reviews.head()

,index,business_id,stars,useful,funny,cool,text,keep
0,2,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,1
1,3,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",1
2,4,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",1
3,5,nlxHRv1zXGT0c0K51q3jDg,5,2,0,0,This is definitely my favorite fast food sub s...,1
4,6,Pthe4qk5xh4n-ef-9bvMSg,5,1,0,0,"Really good place with simple decor, amazing f...",1


In [62]:
reviews.drop(columns=['keep', 'index'], inplace=True)

C:\Users\Luke\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [63]:
reviews.head()

,business_id,stars,useful,funny,cool,text
0,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...
1,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g..."
2,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas..."
3,nlxHRv1zXGT0c0K51q3jDg,5,2,0,0,This is definitely my favorite fast food sub s...
4,Pthe4qk5xh4n-ef-9bvMSg,5,1,0,0,"Really good place with simple decor, amazing f..."


Looking at sum of review data

In [83]:
reviews.groupby(['business_id']).sum()

,stars,useful,funny,cool
business_id,,,,
--1UhMGODdWsrMastO9DZw,118,20,1,16
--6MefnULPED_I942VcFNA,164,95,31,36
--9e1ONYQuAa-CB_Rrw7Tw,7478,1970,1001,1340
--DaPTJW3-tB1vP-PfdTEg,192,18,3,10
--FBCX-N37CMYDfs790Bnw,672,138,63,100
...,...,...,...,...
zzsU528uoRB6qZUGhKDa6w,28,19,15,15
zzvlwkcNR1CCqOPXwuvz2A,24,4,0,1
zzwhN7x37nyjP0ZM8oiHmw,325,62,10,31


Saving progress so importing later is easier.

In [65]:
reviews.to_csv('./yelp_reviews.csv', index=False)

In [66]:
restaurants.to_csv('./yelp_restaurants.csv', index=False)

In [69]:
restaurants.head(2)

,business_id,stars,useful,funny,cool,text
0,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...
1,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g..."


Sorting the values alphabetically by business_id.

In [72]:
reviews.sort_values('business_id', inplace=True)

C:\Users\Luke\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
restaurants.sort_values('business_id', inplace=True)

In [110]:
restaurants

,business_id,name,city,state,stars,review_count,is_open,categories
35086,--1UhMGODdWsrMastO9DZw,The Spicy Amigos,Calgary,AB,4.0,27,1,"Restaurants, Mexican"
27463,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,Richmond Hill,ON,3.0,55,1,"Chinese, Restaurants"
39666,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,Las Vegas,NV,4.0,1759,1,"Seafood, Cajun/Creole, Restaurants, Steakhouses"
9364,--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Toronto,ON,4.0,51,1,"Restaurants, Breakfast & Brunch"
12501,--FBCX-N37CMYDfs790Bnw,The Bar At Bermuda & St. Rose,Henderson,NV,4.0,166,1,"American (New), Nightlife, Beer, Wine & Spirit..."
...,...,...,...,...,...,...,...,...
44452,zzsU528uoRB6qZUGhKDa6w,Joni's Capitol Cafe,Phoenix,AZ,4.0,7,0,"Sandwiches, Breakfast & Brunch, Restaurants"
53996,zzvlwkcNR1CCqOPXwuvz2A,7 Dim Sum,Toronto,ON,3.5,6,0,"Chinese, Asian Fusion, Dim Sum, Restaurants, V..."
4726,zzwhN7x37nyjP0ZM8oiHmw,Carmody's Grille,Pittsburgh,PA,4.0,75,1,"American (New), Restaurants"
31919,zzwicjPC9g246MK2M1ZFBA,Sizzler,Mesa,AZ,3.0,81,1,"Buffets, Salad, American (Traditional), Seafoo..."


The length of the restaurants df (63,961 rows) matches the groupby from above (63,961 rows) so I can easily create composite columns in the restaurant df.  

In [78]:
review_sums = reviews.groupby(['business_id']).sum()

In [112]:
review_sums.head(2)

,useful,funny,cool,total_stars
business_id,,,,
--1UhMGODdWsrMastO9DZw,20,1,16,118
--6MefnULPED_I942VcFNA,95,31,36,164


In [124]:
restaurants['total_stars'] = review_sums['total_stars']
restaurants['total_useful'] = review_sums['useful']
restaurants['total_funny'] = review_sums['funny']
restaurants['total_cool'] = review_sums['cool']

In [115]:
restaurants.shape

(63961, 12)

In [116]:
review_sums.shape

(63961, 4)

In [125]:
restaurants.head()

,index,business_id,name,city,state,stars,review_count,is_open,categories,total_stars,total_useful,total_funny,total_cool
0,35086,--1UhMGODdWsrMastO9DZw,The Spicy Amigos,Calgary,AB,4.0,27,1,"Restaurants, Mexican",118,20,1,16
1,27463,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,Richmond Hill,ON,3.0,55,1,"Chinese, Restaurants",164,95,31,36
2,39666,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,Las Vegas,NV,4.0,1759,1,"Seafood, Cajun/Creole, Restaurants, Steakhouses",7478,1970,1001,1340
3,9364,--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Toronto,ON,4.0,51,1,"Restaurants, Breakfast & Brunch",192,18,3,10
4,12501,--FBCX-N37CMYDfs790Bnw,The Bar At Bermuda & St. Rose,Henderson,NV,4.0,166,1,"American (New), Nightlife, Beer, Wine & Spirit...",672,138,63,100


Reseting indexes so my merge will work.

In [122]:
review_sums.reset_index(inplace=True)

In [123]:
restaurants.reset_index(inplace=True)

In [126]:
restaurants.drop(columns='index', inplace=True)

In [127]:
restaurants.head()

,business_id,name,city,state,stars,review_count,is_open,categories,total_stars,total_useful,total_funny,total_cool
0,--1UhMGODdWsrMastO9DZw,The Spicy Amigos,Calgary,AB,4.0,27,1,"Restaurants, Mexican",118,20,1,16
1,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,Richmond Hill,ON,3.0,55,1,"Chinese, Restaurants",164,95,31,36
2,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,Las Vegas,NV,4.0,1759,1,"Seafood, Cajun/Creole, Restaurants, Steakhouses",7478,1970,1001,1340
3,--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Toronto,ON,4.0,51,1,"Restaurants, Breakfast & Brunch",192,18,3,10
4,--FBCX-N37CMYDfs790Bnw,The Bar At Bermuda & St. Rose,Henderson,NV,4.0,166,1,"American (New), Nightlife, Beer, Wine & Spirit...",672,138,63,100


In [129]:
reviews.reset_index(inplace=True)

In [130]:
reviews.head()

,index,business_id,stars,useful,funny,cool,text
0,2965216,--1UhMGODdWsrMastO9DZw,5,0,0,0,I just love this little Mexican cafeteria-styl...
1,2952308,--1UhMGODdWsrMastO9DZw,3,0,0,0,"We got guacamole, both the vegetarian tacos (R..."
2,2877971,--1UhMGODdWsrMastO9DZw,4,0,0,1,Downtown west which honestly feels like Downto...
3,2580083,--1UhMGODdWsrMastO9DZw,5,0,0,0,Glad these guys moved in on the block! It's qu...
4,2577283,--1UhMGODdWsrMastO9DZw,5,2,1,2,Fantastic spot for lunch with great value for ...


In [131]:
reviews.drop(columns=['index'], inplace=True)

C:\Users\Luke\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [132]:
reviews.shape

(5056227, 6)

In [133]:
reviews.to_csv('yelp_reviews.csv', index=False)

In [134]:
restaurants.to_csv('yelp_restaurants.csv', index=False)

In [135]:
restaurants.head()

,business_id,name,city,state,stars,review_count,is_open,categories,total_stars,total_useful,total_funny,total_cool
0,--1UhMGODdWsrMastO9DZw,The Spicy Amigos,Calgary,AB,4.0,27,1,"Restaurants, Mexican",118,20,1,16
1,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,Richmond Hill,ON,3.0,55,1,"Chinese, Restaurants",164,95,31,36
2,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,Las Vegas,NV,4.0,1759,1,"Seafood, Cajun/Creole, Restaurants, Steakhouses",7478,1970,1001,1340
3,--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Toronto,ON,4.0,51,1,"Restaurants, Breakfast & Brunch",192,18,3,10
4,--FBCX-N37CMYDfs790Bnw,The Bar At Bermuda & St. Rose,Henderson,NV,4.0,166,1,"American (New), Nightlife, Beer, Wine & Spirit...",672,138,63,100


In [137]:
review_sums.to_csv('./review_sums.csv', index=False)